In [1]:
import os
import pickle
import numpy as np
import pandas as pd

In [5]:
def getPositiveNegativeNum(dataframe):
    total_len = len(dataframe)
    positive_num = sum(dataframe.pair_result)
    negative_num = total_len - positive_num
    return (positive_num, negative_num)

def getIndexFromTopByLabel(label, top, dataframe):
    temp = dataframe.where(dataframe.pair_result[:,2] == label).dropna()
    return list(temp.index)[:top]

In [51]:
# version = "v1_5"
# train_set_path = '../data/{0}/train_set_{0}.txt'.format(version)
# train_dataframe = pd.read_csv(train_set_path, sep='\t', header=None)
# sorted_train_dataframe = train_dataframe.sort_values(0).reset_index(drop=True)
# unique_id_list = sorted_train_dataframe[0].unique()
# train_sentence_num_dict = train_dataframe.iloc[:,0].value_counts()
# train_sentence_id_dict = {}

In [52]:
# %%time
# output_train_setence_id_dict_file_path = "../data/{0}/train_set_dict_{0}.pkl".format(version)
# for user_id in unique_id_list:
#     candidate_df = sorted_train_dataframe.where(sorted_train_dataframe[0] == user_id).dropna()
#     train_sentence_id_dict[user_id] = list(candidate_df[1])

# with open(output_train_setence_id_dict_file_path, 'wb') as pickle_file:
#     pickle.dump(train_sentence_id_dict, pickle_file)

In [53]:
# load_train_sentence_dict_path = "../data/{0}/train_set_dict_{0}.pkl".format(version)
# train_sentence_id_dict = None
# with open(load_train_sentence_dict_path, 'rb') as pickle_file:
#     train_sentence_id_dict = pickle.load(pickle_file)

In [13]:
%%time
output_path_prefix = "../data/split_2019_09_08_lemonbear_epoch2_k_100"
output_filtered_split_file_dir = "{0}/filtered_split_result".format(output_path_prefix)
split_num_list = [5,6]
# split_num_list = range(4,7)
k = 100
top = 20
for split_num in split_num_list:
    split_file_path = "{0}/split_result/split_pair_{1}.txt".format(output_path_prefix, split_num)
    split_file = pd.read_csv(split_file_path, sep="\t")
    if not os.path.exists(output_filtered_split_file_dir):
        print("[ERROR] Directory not exists: {0}".format(output_filtered_split_file_dir))
        break
    print("[INFO] Start to process split {0}".format(split_num))
    filtered_index = []
    for i in range(0, len(split_file), k):
        sub_split = split_file.iloc[i:i+k, :]
        first_top_sub_split = split_file.iloc[i:i+top, :]
        first_positive, first_negative = getPositiveNegativeNum(first_top_sub_split)
        all_positive, all_negative = getPositiveNegativeNum(sub_split)
        if first_negative == 0:
            continue
        select_num = min(first_negative, all_positive)
        if select_num == 0:
            continue
        first_negative_index = getIndexFromTopByLabel(0, select_num, first_top_sub_split)
        all_positive_index = getIndexFromTopByLabel(1, select_num, sub_split)
        filtered_index += first_negative_index
        filtered_index += all_positive_index
    if 0 == len(filtered_index):
        print("[WARNING] Filtered num = 0 in split {0}".format(split_num))
        continue
    output_file_path = "{0}/filtered_split_pair_{1}.csv".format(output_filtered_split_file_dir, split_num)
    output_df = split_file.iloc[filtered_index].reset_index(drop=True)
    output_df.to_csv(output_file_path, index=False, sep='\t', header=None)

[INFO] Start to process split 5
[INFO] Start to process split 6
CPU times: user 5min 9s, sys: 7.39 s, total: 5min 17s
Wall time: 5min 14s
